In [ ]:
import pandas as pd;
import os
path='D:/RaniaTarek_JobJourney/Courses content/Data glacier internship/submitted/Data science project_Bank Marketing Campaign/'
files=['bank-additional-full.csv','bank-full.csv','bank-additional.csv','bank.csv']

dfs=[]
for file_name in files:
    
    file=path+file_name
    
    df_current=pd.read_csv(file,delimiter=';');
    df_current['filename']=file_name.split()
    num_columns=len(df_current.columns.tolist())
    column_names=df_current.columns.tolist()
    print('\nInformation about',file_name)

    print('Number of columns= {} '.format(num_columns));

    print('Size of file= {} KB'.format(round(os.stat(file).st_size/1024),2));

    print('Number of duplicated entries= {}'.format(sum(df_current.duplicated())));
    if sum(df_current.duplicated()!=0):
        print('{} duplicates found and deleted'.format(sum(df_current.duplicated())))
        df_current=df_current.drop_duplicates()

    dfs.append(df_current)
    print('Number of unique entries= {}'.format(df_current.size/num_columns));

    
    print('Columns names are {}\n##############################################################'.format(column_names));



In [ ]:
import pandas as pd 
import calendar
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import datetime
df=pd.read_csv('dff_old.csv')

#IMPUTING economic indicators 

xls = pd.ExcelFile('toappend2.xlsx')
print('Sheet names are',xls.sheet_names)

# to read just one sheet to dataframe:
df30_ = pd.read_excel('toappend2.xlsx', sheet_name="daily")
#display(df30_)

df30_["day"] = df30_['Date'].map(lambda x: x.day)
df30_["month"] = df30_['Date'].map(lambda x: x.month)
df30_["month"] = df30_["month"].apply(lambda x: calendar.month_abbr[x].lower())
df30_["year"] = df30_['Date'].map(lambda x: x.year)
df30= df30_.iloc[::-1].reset_index(drop=True)

df['euribor3m'] = 0.0

for i in range(len(df30)):    
    i_raw=df30.iloc[i,:]
    df.loc[(df.day==i_raw.day) & (df.month==i_raw.month) & (df.year==i_raw.year.astype(float)),'euribor3m']=i_raw.Value

df['euribor3m_ffill']=0

for c in ['euribor3m']:
    idx=df[df[c]==float(0)].index
    df.loc[idx,c]=None
    df.loc[idx,'euribor3m_ffill']=1
    display('prefilling',df.loc[idx,c])
df.ffill(axis = 0,inplace=True)

locs=df.loc[df["euribor3m"].isna()]
grp_locs=locs.groupby(['year','month','day']).count()
display('rest',grp_locs)
for l in range(len(grp_locs)):
    d=0
    print(l)
    dayy=locs.loc[l,'day']
    yearr=locs.loc[l,'year']
    monthh=locs.loc[l,'month'].capitalize()
    m=list(calendar.month_abbr).index(monthh)
    orig_d=datetime.datetime(int(yearr),m,dayy)
    print('orid_d',orig_d)
    d=orig_d
    #print(d)
    d-= datetime.timedelta(days=1)
    #print(d)
    found=0
    while(found==0):
        if df30.query("Date==@d").empty:
            d-=datetime.timedelta(days=1)
            print('up')
        else:
            val=df30.query("Date==@d")['Value']
            month_lower=monthh.lower()

            #display(df.query("year==@yearr & month==@month_lower & day==@dayy")['euribor3m'])
            
            print('fill date {} from {} by value {}'.format(orig_d,d,val.values[0]))
            tofill=df.query("year==@yearr & month==@month_lower & day==@dayy").index.tolist()
            print('indicestofill',tofill)
            df.loc[tofill,'euribor3m']=val.values[0]
            display(df.query("year==@yearr & month==@month_lower & day==@dayy")['euribor3m'])


            
            found=1
            
df12_ = pd.read_excel('toappend2.xlsx', sheet_name="monthly")
#Harmonised index of consumer prices, Consumer confidence indicator

df12_.columns
df12_.rename(columns={df12_.columns[0]: 'Date', df12_.columns[1]: 'consum_prices_rate',df12_.columns[2]: 'consum_conf_ind'},inplace=True)
df12_=df12_.iloc[3:]

df12_["day"] = df12_['Date'].map(lambda x: x.day)
df12_["month"] = df12_['Date'].map(lambda x: x.month)
df12_["month"] = df12_["month"].apply(lambda x: calendar.month_abbr[x].lower())
df12_["year"] = df12_['Date'].map(lambda x: x.year)
df12= df12_.iloc[::-1].reset_index(drop=True)

#df12.head()
df['consum_prices_rate'] = 0.0
df['consum_conf_ind'] = 0.0

for i in range(len(df12)):    
    i_raw=df12.iloc[i,:]
    #print(i,i_raw.Value)
    df.loc[(df.day<=i_raw.day) & (df.month==i_raw.month) & (df.year==i_raw.year.astype(float)),'consum_prices_rate']=i_raw.consum_prices_rate
    df.loc[(df.day<=i_raw.day) & (df.month==i_raw.month) & (df.year==i_raw.year.astype(float)),'consum_conf_ind']=i_raw.consum_conf_ind    

df4_ = pd.read_excel('toappend2.xlsx', sheet_name="quaretrly")

df4_.columns
df4_.rename(columns={df4_.columns[0]: 'Date', df4_.columns[1]: 'employed',df4_.columns[2]: 'unemployed',df4_.columns[3]: 'unemployed_rate'},inplace=True)
df4_=df4_.iloc[3:]

df4_["month"] = df4_['Date'].map(lambda x: x.month)
df4_["month"] = df4_["month"].apply(lambda x: calendar.month_abbr[x].lower())
df4_["year"] = df4_['Date'].map(lambda x: x.year)
df4= df4_.iloc[::-1].reset_index(drop=True)

#display(df4.head())
df['employed'] = 0.0
df['unemployed'] = 0.0
df['unemployed_rate']=0.0

for i in range(len(df4)):    
    i_raw=df4.iloc[i,:]
    #print(i,i_raw.Value)
    df.loc[(df.month<=i_raw.month) & (df.year==i_raw.year.astype(float)),'employed']=i_raw.employed
    df.loc[(df.month<=i_raw.month) & (df.year==i_raw.year.astype(float)),'unemployed']=i_raw.unemployed
    df.loc[(df.month<=i_raw.month) & (df.year==i_raw.year.astype(float)),'unemployed_rate']=i_raw.unemployed_rate    

df.to_csv("after_append.csv",
          index=False,
          encoding="utf-8")

# Ambiguous number of entries per feature:
# 288 unknowsn in job, 1857 unknowsn in education, 13020 unkowns in contact , 1840 other in poutcome, 36959 unkowns in poutcome

# poutcome 
ct_table_ind=pd.crosstab(df["poutcome"],df["pdays"]==999)
print('contingency_table between poutcome and pdays==999:\n{}'.format(ct_table_ind))
#change unknown to nonexistant where pdays==999, ow delete row
df.loc[((df['poutcome']=='unknown') & (df["pdays"] == 999)), "poutcome"] = 'non_existant'
indices_to_drop=df.loc[(df['poutcome']=="unknown") & (df['pdays']!=999)].index
df.drop(indices_to_drop,axis=0,inplace=True)
df.reset_index(inplace=True)
#change pdays=999 to pdays=0 to avoid contributing to outliers 
df.loc[df["pdays"] == 999, "pdays"] = 0
df.drop('index',axis=1,inplace=True)
#df
import matplotlib.pyplot as plt

for c in ['job','marital']:
    values=pd.DataFrame()
    norm=pd.DataFrame()

    values['yes']=df[df['y']=='yes'][c].value_counts().sort_values()
    values['no']=df[df['y']=='no'][c].value_counts().sort_values()
    display(values)
    norm['yes']=values['yes']/values.max(axis=1)
    norm['no']=values['no']/values.max(axis=1)

    display(norm)


    norm.plot(kind='bar')
    plt.xlabel(f'{c}')
    plt.ylabel('Number of clients')
    plt.title('{}'.format(c))
    plt.show()
    
df.groupby(['job','education'])['age'].count()#*100/df.query('age<21').groupby(['job'])['age'].count()

df.loc[df.y=="yes",'y_equiv']=int(1)
df.loc[df.y=="no",'y_equiv']=int(0)

#flags for "unknown values" in https://machinelearningmastery.com/binary-flags-for-missing-values-for-machine-learning/

df['contact_missing'] = 0
df['poutcome_missing'] = 0
df['job_missing'] = 0
df['education_missing'] = 0

df.loc[df.contact=="unknown",'contact_missing']=1
df.loc[df.poutcome=="other",'poutcome_missing']=1
df.loc[df.job=="unknown",'job_missing']=1
df.loc[df.education=="unknown",'education_missing']=1

#nominal and cardinal 
#https://pianalytix.com/how-do-you-handle-missing-values-categorical-data-and-feature-scaling-in-machine-learning/
#sefidian.com/2021/07/02/measure-the-correlation-between-numerical-and-categorical-variables-and-the-correlation-between-two-categorical-variables-in-python-chi-square-and-anova/    

####################################################### balance 
df.loc[df["balance"]<0,'overdraft']=1 #this indicates bad credit score so overall not the best candidate to waste time on
df.loc[df["balance"]>=0,'overdraft']=0

df.loc[df["balance"]<0,'balance']=0 #as if balancerepresent +ve balance only
#binning of balance then label_encode
#assume5bgroups
bins_it=(df.balance.max()-df.balance.min())/5
bins=[]#[df.balance.min()]
for i in range(1,5):
    bins.append(bins_it*i+1)

bins.append(df.balance.max())
#bins=[bins_middle,df.balance.max()]
print('bins based on train data',bins)
orig_balance=df['balance']
df['balance'] = pd.cut(df['balance'], bins=5, labels=['vlow','low','med','high','vhigh'])

#make all categoric features numeric ! 
#cats
#job --> hot encoding, marital as well and education as well ? 
#default,housing,load,contact yes=1,no=0, contact=unknown==0(i have another repreesntative)
#month -> numerical
#day_of_week -> 
#fix ranges where numeric variables exist! 

binary=['default','housing','loan','contact','poutcome']
orig_binary=df[binary]

df.loc[df['default']=="yes",'default']=1
df.loc[df['default']=="no",'default']=0

df.loc[df['housing']=="yes",'housing']=1
df.loc[df['housing']=="no",'housing']=0

df.loc[df['loan']=="yes",'loan']=1
df.loc[df['loan']=="no",'loan']=0

df.loc[df['contact']=="cellular",'contact']=1
df.loc[(df['contact'] == "telephone") | (df['contact'] == "unknown"),'contact']=0 #we can cast unknowns as 0 now that we have a variable to indicate it ! 

df.loc[df['poutcome'] =="success",'poutcome']=1
df.loc[(df['poutcome'] == "non_existant") | (df['poutcome'] == "failure") | (df['poutcome'] == "other"),'poutcome']=0


months = {month: index for index, month in enumerate(calendar.month_abbr) if month}
months['Jan']
for month in months:
    #print(month.lower(),months[month])
    df.loc[df['month']==month.lower(),'month']=months[month]

df.head(5)

df.to_csv("before_hotencoding.csv",
          index=False,
          encoding="utf-8")

df['balance']=df.balance.cat.codes

#for high cardinality job according to how frequent a value appears, encode it .. BEFORE THIS STEP, SPLIT DATA FOR TRAINING AND TESTING AND ONLY USE TARGETS OF TRAINING DATA
X_train, X_test, y_train, y_test = train_test_split(df[df.columns[df.columns!='y_equiv']], df['y_equiv'], test_size=0.2, random_state=11)

#https://analyticsindiamag.com/a-complete-guide-to-categorical-data-encoding/
#https://towardsdatascience.com/dealing-with-categorical-variables-by-using-target-encoder-a0f1733a4c69
categories = X_train.job.unique()#df['job'].unique()
targets = y_train.astype('int32').unique()#df['y_equiv'].astype('int32').unique()
cat_list = []
for cat in categories:
    aux_dict = {}
    aux_dict['category'] = cat
    aux_df = X_train[X_train['job'] == cat]
    counts = df.loc[X_train[X_train['job'] == cat].index,'y_equiv'].value_counts()
    aux_dict['count'] = sum(counts)
    for t in targets:
# #        print(t)
        aux_dict['target_' + str(t)] = counts[t]
    cat_list.append(aux_dict)
cat_list = pd.DataFrame(cat_list)
cat_list['genre_encoded_dumb'] = cat_list['target_1'] / cat_list['count']
display(cat_list)
orig_job=cat_list

for cat in cat_list.category:
    educ_equiv=cat_list.loc[cat_list['category']==cat,'genre_encoded_dumb'].tolist()[0]
    #print(cat,educ_equiv)
    df.loc[df['job']==cat,'job']=educ_equiv#.astype('float64')

cat_list.to_csv("education_list.csv",
          index=False,
          encoding="utf-8")

# pd.DataFrame(X_train,X_test)
#y_train
#y_test
train_set=X_train.join(y_train)
train_set.to_csv("train_set.csv", index=True, index_label='idx',encoding="utf-8")

test_set=X_test.join(y_test)
test_set.to_csv("test_set.csv", index=True, index_label='idx', encoding="utf-8")

# #ordinal encoding for education
orig_education=df['education']

ordinal_educ=df.education.unique().tolist()
ordinal_educ_encoded=[3,2,0,1]
display(ordinal_educ)
 
#for col in ["education"]:
#    df[col]=LabelEncoder().fit_transform(df[col])
#    print(LabelEncoder().fit(df[col]).classes_)
#label encoder assigns ['tertiary', 'secondary', 'unknown', 'primary'] as [0:3] hence losing the ordinal idea 
ind=0
for e in ordinal_educ:
    df.loc[df['education']==e,'education']=ordinal_educ_encoded[ind]
    ind=ind+1
    
orig_mairal=df.marital
dum_df = pd.get_dummies(df['marital'], columns=["marital"] )
# merge with main df bridge_df on key values
df = df.join(dum_df)
df.drop(['marital'],axis=1,inplace=True)

df.to_csv("before_hotencoding2.csv",
          index=False,
          encoding="utf-8")

#nominal data hot encoding
#for col in ["day_of_week","marital"]:
orig_dates=df[["day_of_week","year","month"]]
for col in ["day_of_week","year","month"]:    
    #str(col+'_encoded')
    days_enc=pd.DataFrame(OneHotEncoder().fit_transform(df[[col]]).toarray())
    colname=[]
    for day in range(len(days_enc.columns)):#.column:
        #print(day)
        colname.append(str(col)+str(day+1)) 
    
    #display(colname)
    df[colname] = days_enc

df.columns
#orig #df.drop(['day_of_week','year','month','day'],axis=1,inplace=True)
df.drop(['day_of_week'],axis=1,inplace=True)

df.head(10)
df.to_csv("week9.csv",
          index=False,
          encoding="utf-8")

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('before_hotencoding2.csv')
dff=pd.read_csv('week9.csv')
display(df.columns)
#df.describe()
# check the corr with the variables ['age','duration','pdays','campaign','previous']:
#df.corr()


### Personal standing financially: 

personal_info=['age','job','education','balance','overdraft','default', 'housing', 'loan','divorced','married','single']#,'y_equiv']


#########################################################################
indebt=df['default']+df['housing']+df['loan']
df['indebt']=indebt
personal_info.append('indebt')
######################################################################3
df.loc[df['age']>65,'age_grp']='old'
df.loc[df['age']<=25,'age_grp']='young'
df.loc[(df['age']<=65) & (df['age']>25),'age_grp']='adult'

orig_age_grp=df['age_grp']

ordinal_age_grp=df.age_grp.unique().tolist()
ordinal_age_grp_encoded=[1,0,2]
#display(ordinal_age_grp)
 
ind=0
for e in ordinal_age_grp:
    df.loc[df['age_grp']==e,'age_grp']=int(ordinal_age_grp_encoded[ind])
    ind=ind+1
df['age_grp']=df['age_grp'].astype('int64')

personal_info.append('age_grp')

personal_info=['age','age_grp','job','education','balance','overdraft','default', 'housing', 'loan','indebt','divorced','married','single']#,'y_equiv']
df[personal_info].info()


for col in df:
    if df[col].dtype=='O': 
        print("\n{} values--> {}".format(col,set(df[col])));
    else:
        print("\n{} min={} , max={} ".format(col,df[col].min(),df[col].max()));
        
educ_transform=pd.read_csv('education_list.csv')

##hypothesis 1
df.groupby(['age_grp'])[['age','overdraft']].sum()

##hypothesis 2
df.groupby(['indebt','age_grp'])[['housing']].count()*100/df.groupby(['age_grp'])[['housing']].count()

##hypothesis 3
df.groupby(['education','balance'])['age'].count()*100/df.groupby(['balance'])['age'].count()
df.groupby(['age_grp','balance'])['age'].count()*100/df.groupby(['balance'])['age'].count()

##hypothesis 4
hyp_4=df.query('age_grp==2').groupby(['job']).count()[['age']]*100/len(df.query('age_grp==2'))
edu=pd.merge(left=hyp_4, right=educ_transform, left_on='job', right_on='genre_encoded_dumb')[['age','category','genre_encoded_dumb']]

#hypothesis 5
df.groupby(['overdraft','default'])['age'].count()
#display(df[personal_info].corr().default.overdraft)

#hypothesis 6
df.groupby(['indebt','default'])['age'].count()

##hypothesis 7
hyp7=df.groupby(['job'])['age'].min()
pd.merge(left=hyp7, right=educ_transform, left_on='job', right_on='genre_encoded_dumb')[['age','category','genre_encoded_dumb']]
df.groupby(['married']).min().age

##hypothesis 8 new
df.groupby(['balance','default'])['age'].count()*100/df.groupby(['default'])['age'].count()

##hypothesis 9
hyp9=df.query('age<21').groupby(['job','housing'])['age'].count()*100/df.query('age<21').groupby(['job'])['age'].count()
#display(hyp9)
tmps=df.query('age<21').groupby(['job'])['age'].count()*100/df.query('age<21').housing.count()
pd.merge(left=tmps, right=educ_transform, left_on='job', right_on='genre_encoded_dumb')[['category','genre_encoded_dumb']]

hyp10=df.groupby(['job','education'])['age'].count()*100/df.groupby(['job'])['age'].count()
#display(hyp10)
#pd.merge(left=hyp7, right=educ_transform, left_on='job', right_on='genre_encoded_dumb')[['category','genre_encoded_dumb']]
df.groupby(['age_grp'])['overdraft'].sum()#*100/df.query('age<21').groupby(['job'])['age'].count()

### Personal standing financially: 

this_campaign_info=['campaign','contact','contact_missing','day','day_of_week','month','year', 'duration']

# from sklearn.preprocessing import LabelEncoder
# #monday:1
# for col in ["day_of_week"]:
#     a=LabelEncoder().fit(df[col])
#     print(df[col].unique(),list(a.classes_))
#     df[col]=a.transform(df[col])

mapping = {
    'Sunday': 0,
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,

}

df['day_of_week'] = df['day_of_week'].replace(mapping)
df
#df[this_campaign_info].info()


corrmat = df[this_campaign_info].corr(method='pearson')
display(corrmat)
top_corr_features = corrmat.index
plt.figure(figsize=(12,12))
#plot heat map
g=sns.heatmap(df[this_campaign_info][top_corr_features].corr(),annot=True,cmap="RdYlGn")


for c in this_campaign_info:
    values=pd.DataFrame()
    norm=pd.DataFrame()

    values['yes']=df[df['y']=='yes'][c].value_counts().sort_values()
    values['no']=df[df['y']=='no'][c].value_counts().sort_values()
    #display(values)
    norm['yes']=values['yes']/values.max(axis=1)
    norm['no']=values['no']/values.max(axis=1)

    #display(norm)


    norm.plot(kind='bar')
    plt.xlabel(f'{c}')
    plt.ylabel('Number of clients')
    plt.title('{}'.format(c))
    plt.show();

#sns.boxplot(df['duration'])
df.duration.describe()
for col in ['duration']:
    #outliers['col']=col
    upper_lim = df[col].quantile(.99)
    print('{}''s statistics\nupper limit={}\nmax={}\nlen(> upper_lim)={}\n'.format(col,upper_lim,df[col].max(),len(df[df['duration']>upper_lim])))

df['duration']=df['duration']/60
sns.boxplot(df['duration'])

corrmat = df[this_campaign_info].corr(method='pearson')
display(corrmat)

df['duration']
bins_it=(df.duration.max()-df.duration.min())/5
bins=[]#[df.balance.min()]
for i in range(1,5):
    bins.append(bins_it*i+1)

bins.append(df.duration.max())
#bins=[bins_middle,df.balance.max()]
print('equally spaced bins would look like this',bins)

df.duration.describe()

rec_cond= (df['year']==2008 ) | ((df['year']==2009) & (df['month']<6))
no_rec_cond= (df['year']==2010 ) | ((df['year']==2009) & (df['month']>=6))
df.groupby(['year'])['age'].count()

df.loc[rec_cond,'rec']=1
df.loc[no_rec_cond,'rec']=0
this_campaign_info.append('rec')

df.rec=df.rec.astype('int32')
#df.rec.head()
a=df.groupby(['rec'])['duration'].agg(pd.Series.mode)
b=df.groupby(['rec'])['campaign'].mean()
ab=pd.merge(left=a, right=b, left_on=['rec'], right_on=['rec'])

a=df.groupby(['year'])['duration'].agg(pd.Series.mode)
b=df.groupby(['year'])['campaign'].mean()
ab=pd.merge(left=a, right=b, left_on=['year'], right_on=['year'])
ab

cats=['day_of_week','month']
ind=0
for c in cats:
    #c=cats[0]
    #cunique=df[c].unique()
    values=pd.DataFrame()
    values['yes']=df[df['y']=='yes'][c].value_counts().sort_values()
    values['no']=df[df['y']=='no'][c].value_counts().sort_values()
    display(values)
    values.plot(kind='bar')
    plt.xlabel(f'{c}')
    plt.ylabel('Number of clients')
    plt.title('{}'.format(c))
    plt.show();
    
# df.columns
previous_contact=['pdays', 'previous','poutcome','poutcome_missing']


corrmat = df[previous_contact].corr(method='pearson')
#display(corrmat)
top_corr_features = corrmat.index
plt.figure()
g=sns.heatmap(df[previous_contact][top_corr_features].corr(),annot=True,cmap="RdYlGn")

df.pdays.describe()
df.previous.describe()

for col in ['pdays','previous']:
    #outliers['col']=col
    upper_lim = df[col].quantile(.99)
    print('{}''s statistics\nvar={}\nupper limit={}\nmax={}\nlen(> upper_lim)={}\n'.format(col,df[col].var(),upper_lim,df[col].max(),len(df[df['duration']>upper_lim])))
    

print('# of entries where pdays>0={}\n# of entries where pdays=0={}'.format(len(df.query("pdays>0")), len(df.query("pdays==0"))))#.groupby('y_equiv').count()
print('\n# of entries where previous>0={}\n# of entries where previous=0={}'.format(len(df.query("previous>0")), len(df.query("previous==0"))))#.groupby('y_equiv').count()


forppt=['previous','poutcome']
corrmat = df[forppt].corr(method='pearson')
display(corrmat)
top_corr_features = corrmat.index
plt.figure()
g=sns.heatmap(df[forppt][top_corr_features].corr(),annot=True,cmap="RdYlGn")

for c in ['poutcome']:
    values=pd.DataFrame()
    norm=pd.DataFrame()

    values['yes']=df[df['y']=='yes'][c].value_counts().sort_values()
    values['no']=df[df['y']=='no'][c].value_counts().sort_values()
    display(values)
    norm['yes']=values['yes']/values.max(axis=1)
    norm['no']=values['no']/values.max(axis=1)
    
    norm.plot(kind='bar')
    plt.xlabel(f'{c}')
    plt.ylabel('Number of clients')
    plt.title('{}'.format(c))
    plt.show();
    

import matplotlib.pyplot as plt
df.euribor3m.sort_values()

df.groupby(['y','euribor3m']).count()

#plt.plot(df[df['y']=='yes'].euribor3m.sort_values())#.value_counts()
#df.groupby(df[df['y']=='yes'].euribor3m).count().age.sort_values()

filtered=df[df['y']=='yes'].euribor3m.sort_values()
#plt.plot(df.groupby(filtered).count().age)

#plt.plot(df.groupby(filtered).count().age)
x=df.groupby(filtered).count()[['age']].index.tolist() #euribor3m
y=df.groupby(filtered).count()['age'].values.tolist() #counts 
# plt.plot(x,y)
# plt.show()
plt.plot([yi/df.groupby(filtered).count()['age'].max() for yi in y])


filtered=df[df['y']=='no'].euribor3m.sort_values()#/df[df['y']=='no'].euribor3m.sort_values()
x=df.groupby(filtered).count()[['age']].index.tolist() #euribor3m
y=df.groupby(filtered).count()['age'].values.tolist() #counts 
plt.plot([yi/df.groupby(filtered).count()['age'].max() for yi in y])

##################################################

df.columns
eco_indic=['euribor3m','euribor3m_ffill','consum_prices_rate', 'consum_conf_ind', 'employed', 'unemployed','unemployed_rate']

corrmat = df[eco_indic].corr(method='pearson')
display(corrmat)
top_corr_features = corrmat.index
plt.figure()
g=sns.heatmap(df[eco_indic][top_corr_features].corr(),annot=True,cmap="RdYlGn")
df[eco_indic].describe()

dff['age_grp']=df['age_grp'] 
dff['rec']=df['rec']
dff['indebt']=df['indebt']
dff['balance']=df['balance']
#dff['pdays']=df['pdays']
#dff['previous']=df['previous']
dff['duration']=df['duration']

dff.describe()

dff.columns

dff.to_csv("week10_pptEDA.csv",
          index=False,
          encoding="utf-8")

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

df_train=pd.read_csv('train_set.csv') #to get train indices 
df_test=pd.read_csv('test_set.csv') #to get test indices 

df=pd.read_csv('week10_pptEDA.csv')

toremove=['day','month','year']#'pdays','employed','unemployed','year']#,'previous']
df.drop(toremove,axis=1,inplace=True)

train=df.loc[df_train.idx.tolist()]
test=df.loc[df_test.idx.tolist()]
display(df.columns)

non_categoric=['age', 'euribor3m', 'consum_prices_rate', 'consum_conf_ind', 'unemployed_rate','employed','unemployed','pdays','previous','campaign','duration']
targets=['y','y_equiv']
allbutcat=targets+non_categoric
categoric=df.columns[~df.columns.isin(allbutcat)].tolist()
features=df.columns[~df.columns.isin(targets)].tolist()

fig, axs = plt.subplots(nrows=len(non_categoric), ncols=4, figsize=(15, 30))
fig.subplots_adjust(wspace=0.9, hspace=0.9)

df_log=pd.DataFrame()

tocheckoutliers=['age','pdays','previous','campaign','duration']
for n in non_categoric:
    #print(n)
    idx=non_categoric.index(n)
    
    IQR=df[n].quantile(.75)-df[n].quantile(.25)
    lower=df[n].quantile(.25) - (1.5*IQR)# (Q1 - 1.5 * IQR)
    upper=df[n].quantile(.75) + (1.5*IQR)# (Q3 + 1.5 * IQR
    
    info=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(n,df[n].min(),df[n].mean(),df[n].std(),df[n].kurtosis(),upper,df[n].max(),df[n].skew()))    
    
    df[n].hist(ax=axs[idx,0])
    axs[idx,0].set_title(info)
    
    
    
    df_log[n]=np.log2(df[n]-(df[n].min()-1))
    
    IQR=df_log[n].quantile(.75)-df_log[n].quantile(.25)
    lower=df_log[n].quantile(.25) - (1.5*IQR)# (Q1 - 1.5 * IQR)
    upper=df_log[n].quantile(.75) + (1.5*IQR)# (Q3 + 1.5 * IQR)
    
    #print(n,df_log[n].min(),lower,upper,df_log[n].max())
    
    info_log=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(n,df_log[n].min(),df_log[n].mean(),df_log[n].std(),df_log[n].kurtosis(),upper,df_log[n].max(),df_log[n].skew()))    
    
    
    df_log[n].hist(ax=axs[idx,1])
    axs[idx,1].set_title(info_log)

    df.boxplot(column=n,ax=axs[idx,2])


print('Do not touch categoric columns nor as transformation nor scaling\n Do not remove or scale what seems like outliers from economic indicators ')

train_log=df_log.loc[df_train.idx.tolist()]
test_log=df_log.loc[df_test.idx.tolist()]


outliers_tohandle=['campaign','duration']

df_o=pd.DataFrame()

for n in outliers_tohandle:
    idx=non_categoric.index(n)
    robust=RobustScaler(quantile_range=(1,99)).fit(train_log[[n]])#
    df_o[n]=robust.transform(df_log[[n]]).flatten()

    IQR=df_o[n].quantile(.75)-df_log[n].quantile(.25)
    lower=df_o[n].quantile(.25) - (1.5*IQR)# (Q1 - 1.5 * IQR)
    upper=df_o[n].quantile(.75) + (1.5*IQR)# (Q3 + 1.5 * IQR)
    
    info_o=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(n,df_o[n].min(),df_o[n].mean(),df_o[n].std(),df_o[n].kurtosis(),upper,df_o[n].max(),df_o[n].skew()))    
    axs[idx,3].set_title(info_o)

    df_o[n].hist(ax=axs[idx,3])

    
plt.tight_layout()
plt.show()

df_log[outliers_tohandle]=df_o[outliers_tohandle]


df_opt=df[df.columns[~df.columns.isin(non_categoric)]].join(df_log)

# toignore=problematic+non_categoric
# print(toignore)#=problematic+non_categoric

# df_logs[problematic]=df[problematic]

# df_opt=df[df.columns[~df.columns.isin(toignore)]].join(df_logs)


df_opt.drop('pdays',axis=1,inplace=True)
non_categoric.remove('pdays')

print('transformed and scaled statistics of features/n',df_opt[non_categoric].describe())

# ################tc whether to do this step now or after log ????????????????????????????
# train=df_opt.loc[df_train.idx.tolist()]
# test=df_opt.loc[df_test.idx.tolist()]

from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted, check_consistent_length
from sklearn.preprocessing import label_binarize
from sklearn.utils import check_matplotlib_support
from sklearn.base import is_classifier
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import (
    confusion_matrix,
    plot_confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
    roc_auc_score,
    SCORERS
)
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LogisticRegression #baseline for binary classification problems
from sklearn.ensemble import RandomForestClassifier # weak learners 
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold

train=df_opt.loc[df_train.idx.tolist()]
test=df_opt.loc[df_test.idx.tolist()]

X_train=train[train.columns[~train.columns.isin(['y', 'y_equiv','duration'])]].values
y_train=train.loc[:,(train.columns =='y_equiv')].y_equiv.astype('int32')

X_test=test[test.columns[~test.columns.isin(['y', 'y_equiv','duration'])]].values
y_test=test.loc[:,(test.columns =='y_equiv')].y_equiv.astype('int32')

scores = defaultdict(list)


## logistic regression --> scale non-categoric with minmax since non is gaussian except duration which isn't contextually correct to use anyway
#scaling necessary because of penality L2 that normalizes across rows not columns so they need to be relatable 
non_categoric.remove('duration')
df_lr=df_opt
# for i in non_categoric:
#     minmax=MinMaxScaler()
#     minmaxfit=minmax.fit(train[[i]])
#     tmp=minmax.transform(df_opt[i].values.reshape(-1, 1))
# #    info=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(i,df_opt[i].min(),df_opt[i].mean(),df_opt[i].std(),df_opt[i].kurtosis(),df_opt[i].quantile(.99),df_opt[i].max(),df_opt[i].skew()))      
# #     print(info,'\n')
#     df_lr[i]=tmp
features=df_lr.columns[~df_lr.columns.isin(['y', 'y_equiv'])]
for i in features.tolist():
    minmax=MinMaxScaler()
    minmaxfit=minmax.fit(train[[i]].values)
    tmp=minmax.transform(df_opt[i].values.reshape(-1, 1))
#    info=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(i,df_opt[i].min(),df_opt[i].mean(),df_opt[i].std(),df_opt[i].kurtosis(),df_opt[i].quantile(.99),df_opt[i].max(),df_opt[i].skew()))      
#     print(info,'\n')
    df_lr[i]=tmp

    
train_lr=df_lr.loc[df_train.idx.tolist()]
test_lr=df_lr.loc[df_test.idx.tolist()]

X_train_lr=train_lr[train_lr.columns[~train_lr.columns.isin(['y', 'y_equiv','duration'])]].values
y_train_lr=train_lr.loc[:,(train_lr.columns =='y_equiv')].y_equiv.astype('int32')

X_test_lr=test_lr[test_lr.columns[~test_lr.columns.isin(['y', 'y_equiv','duration'])]].values
y_test_lr=test_lr.loc[:,(test_lr.columns =='y_equiv')].y_equiv.astype('int32')
###############################################################

n_jobs = -1
random_state=11

clf=LogisticRegression(random_state=random_state, n_jobs=n_jobs, max_iter=4000)
pipe=Pipeline(steps=[('lr',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
#cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)


# LogisticRegression().get_params().keys()
param_grid = [{
                'lr__penalty': ['l2','none'],
                'lr__class_weight': ['balanced'],
                'lr__solver': ['sag','newton-cg'],
                'lr__multi_class': ['multinomial','ovr'],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)
gs.fit(X_train_lr, y_train_lr)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test_lr)
y_prob = gs.predict_proba(X_test_lr) #model.predict_proba(X_test)[:,1]


scores["Classifier"].append('lr')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_predict))

scores
end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)


score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df  
#y_prob[:,1]=y_predict

## logisitc regression train w/o missing entries , expected to improve 

cols=['contact','poutcome','job','education','euribor3m']

X_miss=train#_lr
locs=[]

for c in cols:
    if c=='euribor3m':
        cmiss=c+'_ffill'
    else:
        cmiss=c+'_missing'
    tmp=X_miss.loc[(X_miss[c]==0) & (X_miss[cmiss]==1)].index.tolist()
    

    locs.append(tmp)

toskiptrain=set(sorted(sum(locs, [])))

print('{} samples from {} training samples have no-missing data'.format(-1*(len(toskiptrain)-len(train_lr)),len(train_lr)))


train_clipped=train_lr.drop(toskiptrain,axis=0)
train_clipped

X_train_clipped=train_clipped[train_clipped.columns[~train_clipped.columns.isin(['y', 'y_equiv','duration'])]].values
y_train_clipped=train_clipped.loc[:,(train_clipped.columns =='y_equiv')].y_equiv.astype('int32')

# clf2=gs.best_estimator_['lr']
# # pipe=Pipeline(StandardScaler(),clf2)

# pipe=make_pipeline(clf2)
# pipe.fit(X_train_clipped, y_train_clipped)  # apply scaling on training data
########################################################################################3

clf=LogisticRegression(random_state=random_state, n_jobs=n_jobs, max_iter=4000)
pipe=Pipeline(steps=[('lr',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
#cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

# LogisticRegression().get_params().keys()
param_grid = [{
                'lr__penalty': ['l2','none'],
                'lr__class_weight': ['balanced'],
                'lr__solver': ['sag','newton-cg'],
                'lr__multi_class': ['multinomial','ovr'],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)
gs.fit(X_train_clipped, y_train_clipped)

##########################################################################################



print('best parameters are',gs.best_params_)

start = time.time()

y_predict=gs.predict(X_test_lr)

y_prob = gs.predict_proba(X_test_lr)

scores["Classifier"].append('lr-missingvalues')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df  

##random forest 

n_jobs = -1
random_state=11

clf=RandomForestClassifier(n_jobs=n_jobs,random_state=random_state)#20 is nothing, std is 100,min_samples_split 5? default=2,min_samples_leafint 3? default=1 

pipe=Pipeline(steps=[('rf',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
#cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.

# LogisticRegression().get_params().keys()
param_grid = [{
                'rf__n_estimators': [150],
                'rf__max_depth':[len(X_train/2),3*len(X_train)/4],
                'rf__criterion': ['gini','entropy'],
                'rf__class_weight': ['balanced', 'balanced_subsample'],
                'rf__oob_score':[True],  
                'rf__max_samples':[0.2,0.4],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train, y_train)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('rf')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)
gs_rf=gs
score_df  

## bagging classifier

n_jobs = -1
random_state=11
from imblearn.ensemble import BalancedBaggingClassifier 
from sklearn.tree import DecisionTreeClassifier
                               
clf3=gs.best_estimator_['rf']
# clf3=score_df.loc['rf'].bestmodel
bag=BalancedBaggingClassifier(base_estimator=clf3,replacement=False,
                                random_state=random_state,
                             n_jobs=n_jobs)

pipe=Pipeline(steps=[('bag',bag)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
# cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC
#right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.

# LogisticRegression().get_params().keys()
param_grid = [{
                'bag__sampling_strategy':['all','not majority','majority'],
}]

gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train, y_train)

print('best parameters are',gs.best_params_)
start = time.time()
y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('bag_rf')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df     

n_jobs = -1
random_state=11

clf=RandomForestClassifier(n_jobs=n_jobs,random_state=random_state)#20 is nothing, std is 100,min_samples_split 5? default=2,min_samples_leafint 3? default=1 

pipe=Pipeline(steps=[('rf',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
# cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.
train_clipped=train.drop(toskiptrain,axis=0)
train_clipped

X_train_clipped=train_clipped[train_clipped.columns[~train_clipped.columns.isin(['y', 'y_equiv','duration'])]].values
y_train_clipped=train_clipped.loc[:,(train_clipped.columns =='y_equiv')].y_equiv.astype('int32')

# LogisticRegression().get_params().keys()
param_grid = [{
                'rf__n_estimators': [150],
                'rf__max_depth':[len(X_train_clipped)/2,3*len(X_train_clipped)/4],
                'rf__criterion': ['gini','entropy'],
                'rf__class_weight': ['balanced', 'balanced_subsample'],
                'rf__oob_score':[True],  
                'rf__max_samples':[0.2,0.4],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train_clipped, y_train_clipped)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('rf_missingvalues')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df

from collections import Counter
counter = Counter(y_train)
display(counter)
# estimate scale_pos_weight value
#estimate = counter[0] / counter[1]
#print('Estimate: %.3f' % estimate)

for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    print(score_name)
    #scores[score_name].append(metric(y_test, y_predict))
    
##xgb boost 

from collections import Counter

counter = Counter(y_train)
display(counter)
# estimate scale_pos_weight value
estimate = counter[0] / counter[1]
print('Estimate: %.3f' % estimate)

from xgboost import XGBClassifier
# weights = [1, 10, 25, 50, 75, 99, 100, 1000]
# param_grid = dict(scale_pos_weight=weights)

n_jobs = -1
random_state=11

clf=XGBClassifier(n_jobs=n_jobs,random_state=random_state)#20 is nothing, std is 100,min_samples_split 5? default=2,min_samples_leafint 3? default=1 

pipe=Pipeline(steps=[ ('xgb',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
# cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.

# LogisticRegression().get_params().keys()
param_grid = [{
    'xgb__scale_pos_weight': [2/estimate,3/estimate,5/estimate,estimate**(1./5),estimate**(1./3),estimate**(1./2),estimate]
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train, y_train)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('xgb')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df  

score_df.to_csv("final_models_results.csv",
          index=False,
          encoding="utf-8")

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

df_train=pd.read_csv('train_set.csv') #to get train indices 
df_test=pd.read_csv('test_set.csv') #to get test indices 

df=pd.read_csv('week10_pptEDA.csv')

toremove=['day','month','year']#'pdays','employed','unemployed','year']#,'previous']
df.drop(toremove,axis=1,inplace=True)

train=df.loc[df_train.idx.tolist()]
test=df.loc[df_test.idx.tolist()]
display(df.columns)

non_categoric=['age', 'euribor3m', 'consum_prices_rate', 'consum_conf_ind', 'unemployed_rate','employed','unemployed','pdays','previous','campaign','duration']
targets=['y','y_equiv']
allbutcat=targets+non_categoric
categoric=df.columns[~df.columns.isin(allbutcat)].tolist()
features=df.columns[~df.columns.isin(targets)].tolist()

fig, axs = plt.subplots(nrows=len(non_categoric), ncols=4, figsize=(15, 30))
fig.subplots_adjust(wspace=0.9, hspace=0.9)

df_log=pd.DataFrame()

tocheckoutliers=['age','pdays','previous','campaign','duration']
for n in non_categoric:
    #print(n)
    idx=non_categoric.index(n)
    
    IQR=df[n].quantile(.75)-df[n].quantile(.25)
    lower=df[n].quantile(.25) - (1.5*IQR)# (Q1 - 1.5 * IQR)
    upper=df[n].quantile(.75) + (1.5*IQR)# (Q3 + 1.5 * IQR
    
    info=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(n,df[n].min(),df[n].mean(),df[n].std(),df[n].kurtosis(),upper,df[n].max(),df[n].skew()))    
    
    df[n].hist(ax=axs[idx,0])
    axs[idx,0].set_title(info)
    
    
    
    df_log[n]=np.log2(df[n]-(df[n].min()-1))
    
    IQR=df_log[n].quantile(.75)-df_log[n].quantile(.25)
    lower=df_log[n].quantile(.25) - (1.5*IQR)# (Q1 - 1.5 * IQR)
    upper=df_log[n].quantile(.75) + (1.5*IQR)# (Q3 + 1.5 * IQR)
    
    #print(n,df_log[n].min(),lower,upper,df_log[n].max())
    
    info_log=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(n,df_log[n].min(),df_log[n].mean(),df_log[n].std(),df_log[n].kurtosis(),upper,df_log[n].max(),df_log[n].skew()))    
    
    
    df_log[n].hist(ax=axs[idx,1])
    axs[idx,1].set_title(info_log)

    df.boxplot(column=n,ax=axs[idx,2])


print('Do not touch categoric columns nor as transformation nor scaling\n Do not remove or scale what seems like outliers from economic indicators ')

train_log=df_log.loc[df_train.idx.tolist()]
test_log=df_log.loc[df_test.idx.tolist()]


outliers_tohandle=['campaign','duration']

df_o=pd.DataFrame()

for n in outliers_tohandle:
    idx=non_categoric.index(n)
    robust=RobustScaler(quantile_range=(1,99)).fit(train_log[[n]])#
    df_o[n]=robust.transform(df_log[[n]]).flatten()

    IQR=df_o[n].quantile(.75)-df_log[n].quantile(.25)
    lower=df_o[n].quantile(.25) - (1.5*IQR)# (Q1 - 1.5 * IQR)
    upper=df_o[n].quantile(.75) + (1.5*IQR)# (Q3 + 1.5 * IQR)
    
    info_o=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(n,df_o[n].min(),df_o[n].mean(),df_o[n].std(),df_o[n].kurtosis(),upper,df_o[n].max(),df_o[n].skew()))    
    axs[idx,3].set_title(info_o)

    df_o[n].hist(ax=axs[idx,3])

    
plt.tight_layout()
plt.show()

df_log[outliers_tohandle]=df_o[outliers_tohandle]


df_opt=df[df.columns[~df.columns.isin(non_categoric)]].join(df_log)

# toignore=problematic+non_categoric
# print(toignore)#=problematic+non_categoric

# df_logs[problematic]=df[problematic]

# df_opt=df[df.columns[~df.columns.isin(toignore)]].join(df_logs)


df_opt.drop('pdays',axis=1,inplace=True)
non_categoric.remove('pdays')

print('transformed and scaled statistics of features/n',df_opt[non_categoric].describe())

# ################tc whether to do this step now or after log ????????????????????????????
# train=df_opt.loc[df_train.idx.tolist()]
# test=df_opt.loc[df_test.idx.tolist()]

from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted, check_consistent_length
from sklearn.preprocessing import label_binarize
from sklearn.utils import check_matplotlib_support
from sklearn.base import is_classifier
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import (
    confusion_matrix,
    plot_confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
    roc_auc_score,
    SCORERS
)
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LogisticRegression #baseline for binary classification problems
from sklearn.ensemble import RandomForestClassifier # weak learners 
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold

train=df_opt.loc[df_train.idx.tolist()]
test=df_opt.loc[df_test.idx.tolist()]

X_train=train[train.columns[~train.columns.isin(['y', 'y_equiv','duration'])]].values
y_train=train.loc[:,(train.columns =='y_equiv')].y_equiv.astype('int32')

X_test=test[test.columns[~test.columns.isin(['y', 'y_equiv','duration'])]].values
y_test=test.loc[:,(test.columns =='y_equiv')].y_equiv.astype('int32')

scores = defaultdict(list)


## logistic regression --> scale non-categoric with minmax since non is gaussian except duration which isn't contextually correct to use anyway
#scaling necessary because of penality L2 that normalizes across rows not columns so they need to be relatable 
# non_categoric.remove('duration')
df_lr=df_opt
# for i in non_categoric:
#     minmax=MinMaxScaler()
#     minmaxfit=minmax.fit(train[[i]])
#     tmp=minmax.transform(df_opt[i].values.reshape(-1, 1))
# #    info=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(i,df_opt[i].min(),df_opt[i].mean(),df_opt[i].std(),df_opt[i].kurtosis(),df_opt[i].quantile(.99),df_opt[i].max(),df_opt[i].skew()))      
# #     print(info,'\n')
#     df_lr[i]=tmp
features=df_lr.columns[~df_lr.columns.isin(['y', 'y_equiv'])]
for i in features.tolist():
    minmax=MinMaxScaler()
    minmaxfit=minmax.fit(train[[i]].values)
    tmp=minmax.transform(df_opt[i].values.reshape(-1, 1))
#    info=str('{}\nmin={:.2f},mean={:.2f},std={:.2f}\nkurtosis={:.2f},outliers={:.2f},max={:.2f}\nskew={:.2f}'.format(i,df_opt[i].min(),df_opt[i].mean(),df_opt[i].std(),df_opt[i].kurtosis(),df_opt[i].quantile(.99),df_opt[i].max(),df_opt[i].skew()))      
#     print(info,'\n')
    df_lr[i]=tmp

    
train_lr=df_lr.loc[df_train.idx.tolist()]
test_lr=df_lr.loc[df_test.idx.tolist()]

X_train_lr=train_lr[train_lr.columns[~train_lr.columns.isin(['y', 'y_equiv'])]].values
y_train_lr=train_lr.loc[:,(train_lr.columns =='y_equiv')].y_equiv.astype('int32')

X_test_lr=test_lr[test_lr.columns[~test_lr.columns.isin(['y', 'y_equiv'])]].values
y_test_lr=test_lr.loc[:,(test_lr.columns =='y_equiv')].y_equiv.astype('int32')
###############################################################

n_jobs = -1
random_state=11

clf=LogisticRegression(random_state=random_state, n_jobs=n_jobs, max_iter=4000)
pipe=Pipeline(steps=[('lr',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
#cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)


# LogisticRegression().get_params().keys()
param_grid = [{
                'lr__penalty': ['l2','none'],
                'lr__class_weight': ['balanced'],
                'lr__solver': ['sag','newton-cg'],
                'lr__multi_class': ['multinomial','ovr'],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)
gs.fit(X_train_lr, y_train_lr)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test_lr)
y_prob = gs.predict_proba(X_test_lr) #model.predict_proba(X_test)[:,1]


scores["Classifier"].append('lr')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_predict))

scores
end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)


score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df  
#y_prob[:,1]=y_predict

## logisitc regression train w/o missing entries , expected to improve 

cols=['contact','poutcome','job','education','euribor3m']

X_miss=train#_lr
locs=[]

for c in cols:
    if c=='euribor3m':
        cmiss=c+'_ffill'
    else:
        cmiss=c+'_missing'
    tmp=X_miss.loc[(X_miss[c]==0) & (X_miss[cmiss]==1)].index.tolist()
    

    locs.append(tmp)

toskiptrain=set(sorted(sum(locs, [])))

print('{} samples from {} training samples have no-missing data'.format(-1*(len(toskiptrain)-len(train_lr)),len(train_lr)))


train_clipped=train_lr.drop(toskiptrain,axis=0)
train_clipped

X_train_clipped=train_clipped[train_clipped.columns[~train_clipped.columns.isin(['y', 'y_equiv'])]].values
y_train_clipped=train_clipped.loc[:,(train_clipped.columns =='y_equiv')].y_equiv.astype('int32')

# clf2=gs.best_estimator_['lr']
# # pipe=Pipeline(StandardScaler(),clf2)

# pipe=make_pipeline(clf2)
# pipe.fit(X_train_clipped, y_train_clipped)  # apply scaling on training data
########################################################################################3

clf=LogisticRegression(random_state=random_state, n_jobs=n_jobs, max_iter=4000)
pipe=Pipeline(steps=[('lr',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
#cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

# LogisticRegression().get_params().keys()
param_grid = [{
                'lr__penalty': ['l2','none'],
                'lr__class_weight': ['balanced'],
                'lr__solver': ['sag','newton-cg'],
                'lr__multi_class': ['multinomial','ovr'],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)
gs.fit(X_train_clipped, y_train_clipped)

##########################################################################################



print('best parameters are',gs.best_params_)

start = time.time()

y_predict=gs.predict(X_test_lr)

y_prob = gs.predict_proba(X_test_lr)

scores["Classifier"].append('lr-missingvalues')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test_lr, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df  

##random forest 

n_jobs = -1
random_state=11

clf=RandomForestClassifier(n_jobs=n_jobs,random_state=random_state)#20 is nothing, std is 100,min_samples_split 5? default=2,min_samples_leafint 3? default=1 

pipe=Pipeline(steps=[('rf',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
#cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.

# LogisticRegression().get_params().keys()
param_grid = [{
                'rf__n_estimators': [150],
                'rf__max_depth':[len(X_train/2),3*len(X_train)/4],
                'rf__criterion': ['gini','entropy'],
                'rf__class_weight': ['balanced', 'balanced_subsample'],
                'rf__oob_score':[True],  
                'rf__max_samples':[0.2,0.4],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train, y_train)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('rf')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)
gs_rf=gs
score_df  

## bagging classifier

n_jobs = -1
random_state=11
from imblearn.ensemble import BalancedBaggingClassifier 
from sklearn.tree import DecisionTreeClassifier
                               
clf3=gs.best_estimator_['rf']
# clf3=score_df.loc['rf'].bestmodel
bag=BalancedBaggingClassifier(base_estimator=clf3,replacement=False,
                                random_state=random_state,
                             n_jobs=n_jobs)

pipe=Pipeline(steps=[('bag',bag)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
# cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC
#right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.

# LogisticRegression().get_params().keys()
param_grid = [{
                'bag__sampling_strategy':['all','not majority','majority'],
}]

gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train, y_train)

print('best parameters are',gs.best_params_)
start = time.time()
y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('bag_rf')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df     

n_jobs = -1
random_state=11

clf=RandomForestClassifier(n_jobs=n_jobs,random_state=random_state)#20 is nothing, std is 100,min_samples_split 5? default=2,min_samples_leafint 3? default=1 

pipe=Pipeline(steps=[('rf',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
# cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.
train_clipped=train.drop(toskiptrain,axis=0)
train_clipped

X_train_clipped=train_clipped[train_clipped.columns[~train_clipped.columns.isin(['y', 'y_equiv'])]].values
y_train_clipped=train_clipped.loc[:,(train_clipped.columns =='y_equiv')].y_equiv.astype('int32')

# LogisticRegression().get_params().keys()
param_grid = [{
                'rf__n_estimators': [150],
                'rf__max_depth':[len(X_train_clipped)/2,3*len(X_train_clipped)/4],
                'rf__criterion': ['gini','entropy'],
                'rf__class_weight': ['balanced', 'balanced_subsample'],
                'rf__oob_score':[True],  
                'rf__max_samples':[0.2,0.4],
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train_clipped, y_train_clipped)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('rf_missingvalues')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df

from collections import Counter
counter = Counter(y_train)
display(counter)
# estimate scale_pos_weight value
#estimate = counter[0] / counter[1]
#print('Estimate: %.3f' % estimate)

for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    print(score_name)
    #scores[score_name].append(metric(y_test, y_predict))
    
##xgb boost 

from collections import Counter

counter = Counter(y_train)
display(counter)
# estimate scale_pos_weight value
estimate = counter[0] / counter[1]
print('Estimate: %.3f' % estimate)

from xgboost import XGBClassifier
# weights = [1, 10, 25, 50, 75, 99, 100, 1000]
# param_grid = dict(scale_pos_weight=weights)

n_jobs = -1
random_state=11

clf=XGBClassifier(n_jobs=n_jobs,random_state=random_state)#20 is nothing, std is 100,min_samples_split 5? default=2,min_samples_leafint 3? default=1 

pipe=Pipeline(steps=[ ('xgb',clf)])

cv= RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=random_state)
# cv = StratifiedKFold(shuffle=True, n_splits=6, random_state=random_state)

## ince you are using scikit, try gradient boosted trees on your data. You'll probably get better precision-recall AUC right out of the box. SVCs, as you point out, are not really practical for anything but very small datasets.

# LogisticRegression().get_params().keys()
param_grid = [{
    'xgb__scale_pos_weight': [2/estimate,3/estimate,5/estimate,estimate**(1./5),estimate**(1./3),estimate**(1./2),estimate]
}]


gs=GridSearchCV(pipe, param_grid, scoring='precision',cv=cv)

gs.fit(X_train, y_train)

print('best parameters are',gs.best_params_)
start = time.time()

y_predict=gs.predict(X_test)
y_prob = gs.predict_proba(X_test) #model.predict_proba(X_test)[:,1]

#scores = defaultdict(list)

scores["Classifier"].append('xgb')

for metric in [log_loss]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_prob[:, 1]))


for metric in [accuracy_score,precision_score, recall_score, f1_score, roc_auc_score,r2_score]:#,calibration_curve_ece]:
    score_name = metric.__name__.replace("_", " ").replace("score", "").capitalize()
    scores[score_name].append(metric(y_test, y_predict))

end = time.time()
scores['time'].append(end-start)
scores['bestmodel'].append(gs.best_params_)

score_df = pd.DataFrame(scores).set_index("Classifier")
score_df.round(decimals=3)

score_df  

score_df.to_csv("final_models_results_duration.csv",
          index=False,
          encoding="utf-8")